# Comparison with real time data

Lets take this to the next level - we will train a model and then use it to try and predict the current taxi availability.

In [14]:
from sgtaxidata import *
from sklearn.ensemble import *
import numpy as np
from datetime import datetime
import time


DBNAME="../taxis.db"

holidays=['20170626']
def peak(h,m,w):
    return 1 if ((w==1 and (h>=6 and (h <=9 and m<30)) or (h >= 18))) else 0

def weekday(d):
    dt=datetime.strptime(d,"%Y%m%d")
    return 1 if datetime.isoweekday(dt)<6 else 0

In [21]:
lat,lon = utils.get_lat_long("Sengkang")
dates=taxis.get_dates(DBNAME)
data=[analysis.get_weekday_data(date,DBNAME,lat,lon,holidays=holidays) for date in dates]
data=[x for x in data if x is not None]

In [22]:
rf=RandomForestRegressor(n_estimators=100,n_jobs=10,warm_start=True)
for df in data:
    X=df.drop(['TaxiCount','PeakHour'],axis=1)
    y=np.array(df['TaxiCount'])
    rf.fit(X,y.ravel())
    rf.n_estimators+=20

In [ ]:
h=20
grid=utils.create_grid(h)
d=utils.get_taxi_data()
points=d['features'][0]['geometry']['coordinates']
for b,a in points:
    i,j=utils.get_grid_cell(a,b,h)
    grid[j][i]+=1

a,b=utils.get_grid_cell(lat,lon)

results=[]
for i in range(10):
    n=datetime.now()
    h=n.hour
    m=int(5 * round(float(n.minute)/5))
    p=1 if datetime.isoweekday(n)<6 else 0
    X=np.array([h,m]).reshape(1,-1)
    results.append(rf.predict(X))
    time.sleep(30000)

visualize.plot_data([(range(10),results),(range(10),[grid[b][a]]*10)])